# Exercise - Forward Pricing and Carry

---

# 1. Forward Pricing Fundamentals

### Data

Consider a Treasury note with the following characteristics:

| Field | Value |
|-------|-------|
| Coupon | 4.00% |
| Maturity | 2030-02-28 |
| Issue Date | 2023-02-28 |
| First Coupon | 2023-08-31 |

Market conditions as of **2023-04-18**:

| Field | Value |
|-------|-------|
| Spot Price | 102-02 (i.e., 102 + 2/32) |
| Repo Rate | 4.85% |
| Forward Date | 2023-08-01 |

In [32]:
import pandas as pd
import numpy as np
from datetime import date

In [33]:
# Bond specifications
date_issue = date(2023, 2, 28)
date_maturity = date(2030, 2, 28)
date_coupon = date(2023, 8, 31)
cpn = 0.04
face = 100

# Market conditions
date_now = date(2023, 4, 18)
date_fwd = date(2023, 8, 1)
spot_price = 102 + 2/32
repo_rate = 0.0485

### 1.1 Accrued Interest

Calculate the accrued interest at:
- Time $t$ (the current date)
- Time $T$ (the forward date)

#### Tip: Accrued Interest Formula

[...]

where $N$ is the face value (100).

In [34]:
time_t = date_now
time_T = date_fwd

accured_t = cpn * face / 2 * ((time_t - date_issue).days) / (date_coupon - date_issue).days
accured_T = cpn * face / 2 * ((time_T - date_issue).days) / (date_coupon - date_issue).days

print(accured_t, accured_T)

0.532608695652174 1.673913043478261


### 1.2 Forward Price

Calculate the forward price of the bond for delivery at time $T$.

In [35]:
accured_t = 0.5326
accured_T = 1.6739

act_360 = (time_T - time_t).days / 360

def forward_price(accured_t, accured_T):
    return (spot_price + accured_t) * (1 + repo_rate * act_360) - accured_T

forward_price(accured_t, accured_T)


102.37249318541666

### 1.3 The Forward Drop

Calculate the **forward drop**, which is the difference between the spot and forward prices.

Is the forward price higher or lower than the spot? Explain why, in terms of the carry relationship.

In [36]:
spot = spot_price
fwd = forward_price(accured_t, accured_T) 
forward_drop = spot - fwd

print("Forward Drop:", forward_drop, "\n")
print("Forward Price:", fwd)
print("Spot Price:", spot)

Forward Drop: -0.30999318541665843 

Forward Price: 102.37249318541666
Spot Price: 102.0625


# - - -

Forward price is higher than spot price. 

The forward price reflects two effects:

Financing cost (negative carry):
- Repo rate (4.85%) > Coupon rate (4%)
- Financing the bond costs more than the coupon income

Accrued interest growth (positive effect):
- Accrued interest grows from 0.5326 to 1.6739
- This pushes the forward price up

In this case the accured interest outpaces the financing cost so forward is higher than spot.

# - - -

---

# 2. Carry Analysis

### 2.1 Calculate Carry

Compute the **carry** of holding this bond from $t$ to $T$.

#### Tip: Carry Formula

In [40]:
price_dirty = spot_price + accured_t

def carry_calc():
    return (cpn * face - repo_rate * price_dirty) * act_360

carry = carry_calc()


### 2.2 Verify the Relationship

Verify that the forward drop approximately equals the carry:


Why might this relationship not hold exactly?

In [ ]:
print("Carry:       ", carry)
print("Forward Drop:", forward_drop)

Carry:        -0.28462651875
Forward Drop: -0.30999318541665843


# - - -

This might not hold because of compounding that is attributed to the forward price computation. Also the carry is a linear approximation.

# - - -

---

# 3. Forward Pricing with Interim Coupon

Now consider a forward contract that spans across a coupon date.

**New scenario:**
- Current date: 2023-04-18
- Forward date: 2023-10-15 (after the August coupon)
- Coupon payment date: 2023-08-31

The bond still has:
- Coupon: 4%
- Spot price: 102-02
- Repo rate: 4.85%

In [48]:
# New forward date (after coupon)
date_fwd_new = date(2023, 10, 15)
date_next_coupon = date(2023, 8, 31)


### 3.1 Forward Price with Interim Coupon

Calculate the forward price when there is an interim coupon payment.

In [59]:
act_360_t0 = (date_next_coupon - date_now).days / 360
act_360_t1 = (date_fwd_new - date_next_coupon).days / 360

time_t = date_now
time_T = date_fwd_new

date_next_coupon_after = date(2024, 2, 28)

accured_t = cpn * face / 2 * ((time_t - date_issue).days) / (date_coupon - date_issue).days
accured_T = cpn * face / 2 * ((time_T - date_next_coupon).days) / (date_next_coupon_after - date_next_coupon).days

print(accured_t)
print(accured_T)

def forward_price_interim(accured_t, accured_T):
    return ((spot_price + accured_t) * (1 + repo_rate * act_360_t0) - cpn * face / 2) * (1 + repo_rate * act_360_t1) - accured_T

forward_price_interim(accured_t, accured_T)

0.532608695652174
0.4972375690607735


102.5849898254811

### 3.2 Compare Forward Prices

How does the forward price with an interim coupon compare to a hypothetical forward price calculated without accounting for the coupon? 

Explain the economic intuition.

# - - -

Accounting for the coupon reduces the financing requirements for the second period as they would be reduced by the coupon payment. And since higher financing = lower forward price, ignoring the coupon would lower the price.

# - - -

---

# 4. Forward vs Futures

### 4.1 Daily Settlement Impact

Suppose the Treasury futures contract on this bond is trading at 101.50.

Given your calculated forward price, is the futures price higher or lower than the forward price?

Explain why Treasury futures prices are typically **lower** than forward prices due to the daily settlement (mark-to-market) mechanism.

# - - -
102.58 forward
101.50 future

Future price is lower due to reinvesement risk after daily settlements as the loses must be additionally funded daily.The daily settlement forces reinvestment at uncertain rates, making futures less attractive than forwards.
# - - -


### 4.2 Tailing the Hedge

If you want to hedge a $10 million notional forward position using futures, how many futures contracts should you use?

#### Tip: Tailing the Hedge

$$N^{\text{fut}} = Z(t,T) \cdot N^{\text{fwd}}$$

Assume each futures contract has a notional of $100,000.

In [ ]:
def Z(t, T):
    act_360 = (T - t).days / 360
    return 1 / (1 + repo_rate * act_360)

N_fut = Z(date_now, date_fwd) * 10000000/100000
print(N_fut)

from math import ceil
print(ceil(N_fut))

98.60514801043571
99


***